# Ambiente

## Instalar Librerías

In [1]:
# pip install --user -r ./requirements.txt -q

In [2]:
# pip show langchain

In [3]:
# pip install --upgrade langchain -q

## Imports

In [4]:
import pandas as pd
from trulens_eval import Tru
from trulens_eval import Select
from trulens_eval.tru_virtual import VirtualApp, VirtualRecord, TruVirtual
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.feedback.feedback import Feedback

In [5]:
model_embedding = 'text-embedding-3-large'
# model_embedding = 'text-embedding-3-small'

model_llm = "gpt-3.5-turbo"
# model_llm = 'gpt-4o'

In [6]:
df_llm = pd.read_csv(f'métricas_llm/métricas_llm_{model_embedding}_{model_llm}.csv', sep=',')
df_llm.head()

,Unnamed: 0,question,contexts,answer,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall,context_relevancy
0,0,como entrar portal online servicios?,['🌟 ¡Bienvenido a tu portal de pago en línea! ...,"Para acceder al portal de servicios en línea, ...",🌟 ¡Hola! Estamos encantados de que estés aquí....,0.906041,0.857143,0.915413,1.00,0.012821
1,1,¿Cómo hacer para usar Servicios en Internet?,"['Para iniciar sesión con SafeKey, primero ase...",Para acceder a los diferentes servicios en int...,🌟 ¡Hola! Estamos encantados de que estés aquí....,0.771379,0.857143,0.881518,0.00,0.010526
2,2,¿Cómo se accede al portal de servicios de la UVM?,['🌟 ¡Hola! Estamos encantados de que estés aqu...,"Para acceder al portal de servicios de la UVM,...",🌟 ¡Hola! Estamos encantados de que estés aquí....,0.928263,1.000000,0.996834,1.00,0.011364
3,3,¿Qué información se necesita para iniciar sesi...,"['Para iniciar sesión por primera vez, simplem...","Para iniciar sesión en el portal, se necesita ...",🌟 ¡Hola! Estamos encantados de que estés aquí....,0.875546,0.800000,0.972100,0.50,0.069444
4,4,Como consutar calificaciones?,['🌟 ¡Hola! Estamos emocionados de poder ayudar...,"Para consultar tus calificaciones, primero deb...",🌟 ¡Hola! Estamos emocionados de poder ayudarte...,1.000000,1.000000,0.934962,0.75,0.010417


In [10]:
# Configuración del VirtualApp
virtual_app_config = {
    "llm": {
        "modelname": model_llm  
    },
    "template": """Responde la siguiente pregunta de manera completa y basándote solo en el contexto proporcionado:

    <contexto>
    {context}
    </contexto>

    Pregunta: {input}"""
    }

virtual_app = VirtualApp(virtual_app_config)
virtual_app[Select.RecordCalls.llm.maxtokens] = 1024

retriever = Select.RecordCalls.retriever
synthesizer = Select.RecordCalls.synthesizer

virtual_app[retriever] = "retriever"
virtual_app[synthesizer] = "synthesizer"

# Crear VirtualRecords desde el DataFrame
data = []

for index, row in df_llm.iterrows():
    question = row['question']
    answer = row['answer']
    context = row['contexts']
    ground_truth = row['ground_truth']

    context_call = retriever.get_context
    generation = synthesizer.generate

    record = VirtualRecord(
        main_input=question,
        main_output=answer,
        calls={
            context_call: dict(
                args=[question],
                rets=[context]
            ),
            generation: dict(
                args=[f"""
                Hemos proporcionado el siguiente contexto: \n
                ---------------------\n
                {context}
                ---------------------\n
                Dada esta información, por favor responde la pregunta: 
                {question}
                """],
                rets=[answer]
            )
        }
    )

    data.append(record)

# Inicializar feedbacks
provider = OpenAI()

context_call = retriever.get_context

f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons,name="Context relevance")
    .on_input()
    .on(context_call.rets[:])
)
# Definir las funciones de retroalimentación para groundedness y answer_relevance
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons,name="Groundedness")
    .on(retriever.get_context.rets[:])
    .on(synthesizer.generate.rets[:])
)

f_answer_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,name="Answer Relevance")
    .on_input()
    .on(synthesizer.generate.rets[:])
)


✅ In Context relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context relevance, input context will be set to __record__.app.retriever.get_context.rets[:] .
✅ In Groundedness, input source will be set to __record__.app.retriever.get_context.rets[:] .
✅ In Groundedness, input statement will be set to __record__.app.synthesizer.generate.rets[:] .
✅ In Answer Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input context will be set to __record__.app.synthesizer.generate.rets[:] .


In [11]:
dir(provider)

['DEFAULT_MODEL_ENGINE',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__'

In [12]:
# Inicializar TruVirtual y agregar registros
virtual_recorder = TruVirtual(
    app_id="model gpt4o embsmallfinal2",
    app=virtual_app,
    feedbacks=[f_context_relevance,f_groundedness,f_answer_relevance],
    feedback_mode="deferred"
)

for record in data:
    virtual_recorder.add_record(record)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [13]:
tru = Tru()

tru.run_dashboard(force=True)
tru.start_evaluator()

tru.stop_evaluator() 

Force stopping dashboard ...
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…